<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">

# Plotting and Visualization

At the end of this lesson you will learn:

1. how to use xarray's convenient matplotlib-backed plotting interface to
   visualize your datasets.
2. that `hvplot` provides an equally convenient interface for bokeh-backed plots

## Table of contents

1. [Basic plotting](#basic)
1. [Histograms](#hist)
1. [2D plots](#2d)
1. [1D line plots](#1d)
1. [Faceting or multiple subplots](#facet)
1. [Geography: matplotlib and cartopy](#geo)
1. [Interactive bokeh plots using hvplot](#bokeh)


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [ ]:
xr.set_options(display_style="html")

## Load data

First lets load up a tutorial dataset to visualize.


In [ ]:
ds = xr.tutorial.open_dataset("air_temperature.nc").rename({"air": "Tair"})

# we will add a gradient field with appropriate attributes
ds["dTdx"] = ds.Tair.differentiate("lon") / 110e3 / np.cos(ds.lat * np.pi / 180)
ds["dTdy"] = ds.Tair.differentiate("lat") / 105e3
ds.dTdx.attrs = {"long_name": "$∂T/∂x$", "units": "°C/m"}
ds.dTdy.attrs = {"long_name": "$∂T/∂y$", "units": "°C/m"}

ds

This dataset has three "data variables", `Tair` is air temperature and `dTdx`
and `dTdy` are horizontal gradients of this temperature field. All three "data
variables" are three-dimensional with dimensions `(time, lat, lon)`.


<a id='basic'></a>

## Basic plotting: .plot()

DataArray objects have a `plot` method. This method creates plots using
`matplotlib` so all of your existing matplotlib knowledge carries over!

By default `.plot()` makes

1. a line plot for 1-D arrays using `plt.plot()`
2. a `pcolormesh` plot for 2-D arrays using `plt.pcolormesh()`
3. a histogram for everything else using `plt.hist()`


<a id='hist'></a>

## Histograms

`Tair` is three-dimensional, so we got a histogram of temperature values. Notice
the label on the x-axis. One of xarray's convenient plotting features is that it
uses the `attrs` of `Tair` to nicely label axes and colorbars.


In [ ]:
ds.Tair.plot()

You can pass extra arguments to the underlying `hist()` call. See the matplotlib
docs (https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.hist.html) for
all possible keyword arguments.

**Tip:** Note that the returned values are exactly what matplotlib would return

### Exercise

Update the above plot to show 50 bins with unfilled steps instead of filled
bars.


<a id='2d'></a>

## 2D plots

Now we will explore 2D plots. Let's select a single timestep of `Tair` to
visualize.


In [ ]:
ds.Tair.isel(time=1).plot()

Notice how much information is on that plot!

The x- and y-axes are labeled with full names — "Latitude", "Longitude" — along
with units. The colorbar has a nice label, again with units. And the title tells
us the timestamp of the data presented.

`plot` takes many keyword arguments and is quite sophisticated (see
https://xarray.pydata.org/en/stable/generated/xarray.plot.pcolormesh.html).

Here is a more complicated figure that explicitly sets `time` as the x-axis,
customizes the colorbar, and overlays two contours at specific levels.

**Tip:** Other options for 2D plots include `.plot.contour`, `.plot.contourf`,
`.plot.imshow`


In [ ]:
ds.Tair.isel(lon=1).plot(
    x="time",  # coordinate to plot on the x-axis of the plot
    robust=True,  # set colorbar limits to 2nd and 98th percentile of data
    cbar_kwargs={
        "orientation": "horizontal",
        "label": "custom label",
        "pad": 0.2,
    },  # passed to plt.colorbar
)

### Exercise

Update the above plot to use a different matplotlib colormap.


### Exercise

Now overlay a contour plot on top of the previous plot


<a id='1d'></a>

## 1D line plots

xarray is also able to plot lines by wrapping `plt.plot()`. As in the earlier
examples, the axes are labelled and keyword arguments can be passed to the
underlying `matplotlib` call.


In [ ]:
ds.Tair.isel(time=1, lon=10).plot(marker="o")

Lets say we want to compare line plots of temperature at three different
latitudes. We can use the `hue` kwarg to do this.


In [ ]:
ds.Tair.isel(time=1).sel(lat=[40, 50, 60], method="nearest").plot(x="lon", hue="lat")

## Customization

All of xarray's plotting functions take an large list kwargs that customize
behaviour. A full list can be seen here:
https://xarray.pydata.org/en/stable/generated/xarray.plot.pcolormesh.html. That
said xarray does not wrap all matplotlib functionality.

The general strategy for making plots that are more complicated that the
examples above is

1. Create a matplotlib axis `ax`
2. Use xarray to make a close approximation of the final plot specifying
   `ax=ax`.
3. Use `ax` methods to fully customize the plot


<a id='facet'></a>

## Faceting

Faceting is the art of presenting "small multiples" of the data. It is an
effective way of visualizing variations of 3D data where 2D slices are
visualized in a panel (subplot) and the third dimensions is varied between
panels (subplots).

Here is where xarray really augments matplotlib's functionality. We will use
monthly means to illustrate


In [ ]:
monthly_means = ds.groupby("time.month").mean()
# xarray's groupby reductions drop attributes. Let's assign them back so we get nice labels.
monthly_means.Tair.attrs = ds.Tair.attrs
monthly_means

Note that the dimensions are now `lat, lon, month`.

We want to visualize how the monthly mean air temperature varies with month of
the year.

The simplest way to facet is to specify the `row` or `col` kwargs which are
expected to be a dimension name. Here we use `month` so that each panel or
"facet" of the plot presents the mean temperature field in a given month. Since
a 12 column plot would be too small to interpret, we can "wrap" the facets into
multiple rows using `col_wrap`


In [ ]:
fg = monthly_means.Tair.plot(
    col="month",
    col_wrap=4,  # each row has a maximum of 4 columns
)

All the usual customizations are possible


In [ ]:
fg = monthly_means.Tair.plot(
    col="month",
    col_wrap=4,
    # The remaining kwargs customize the plot just as for not-faceted plots
    robust=True,
    cmap=mpl.cm.RdYlBu_r,
    cbar_kwargs={
        "orientation": "horizontal",
        "shrink": 0.8,
        "aspect": 40,
        "pad": 0.1,
    },
)

The returned FacetGrid object `fg` has many useful properties and methods e.g.

1. `fg.fig` provides a handle to the figure
2. `fg.axes` is a numpy object array with handles to each individual axes
3. `fg.set_xlabels` and `fg.set_ylabels` can be used to change axes labels.

See https://xarray.pydata.org/en/stable/api.html#faceting for a full list.

### Exercise

Use these methods to set a title for the figure using `suptitle`, as well as
change the x- and y-labels.


In [ ]:
fg

### Modifying all facets of a plot

The FacetGrid object has some more advanced methods that let you customize the
plot further.

Here we illustrate the use of `map` and `map_dataarray` that let you map custom
plotting functions to an existing `FacetGrid`. The functions passed to `map` and
`map_dataarray` must have a particular signature. See the docstring for more
details.

Alternatively one can loop over `fg.axes` and modify each individual subplot as
needed


In [ ]:
fg = monthly_means.Tair.plot(col="month", col_wrap=4)

# Use this to plot contours on each panel
# Note that this plotting call uses the original DataArray gradients
fg.map_dataarray(xr.plot.contour, x="lon", y="lat", colors="k", levels=13, add_colorbar=False)

# Add a point (or anything else!)
fg.map(lambda: plt.plot(250, 40, markersize=20, marker=".", color="w"))

### Faceting tip: Use faceting to plot multiple DataArrays

Faceting can be used to plot multiple DataArrays in a Dataset. The trick is to
use `to_array()` to convert a Dataset to a DataArray and then facet that.

This trick only works when it is sensible to use the same colormap and color
scale for all DataArrays like with `dTdx` and `dTdy`


In [ ]:
gradients = monthly_means[["dTdx", "dTdy"]].to_array("gradient")
gradients

In [ ]:
fg = gradients.isel(month=slice(None, None, 3)).plot.contourf(
    levels=13,
    col="month",
    row="gradient",
    robust=True,
    cmap=mpl.cm.coolwarm,
    cbar_kwargs={
        "orientation": "horizontal",
        "shrink": 0.8,
        "aspect": 40,
        "label": "Gradient [°C/m]",
    },
)

<a id="geo"></a>

## Geography: Matplotlib and Cartopy

Since xarray's default plotting functionality builds on matplotlib, we can
seamlessly use cartopy to make nice maps:

1. Specify a `projection` for the plot when creating a new figure `fig` with
   axis `axis`.
2. Explicitly ask xarray to plot to axis `axis` by passing the kwarg `ax=axis`.
3. Specify the projection of the data using `transform` (`PlateCarree` here) in
   `.plot()`.


In [ ]:
import cartopy.crs as ccrs

fig, axis = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Orthographic(-90, 30)))

ds.Tair.isel(time=1).plot(
    ax=axis,
    transform=ccrs.PlateCarree(),  # this is important!
    # usual xarray stuff
    cbar_kwargs={"orientation": "horizontal", "shrink": 0.7},
    robust=True,
)
axis.coastlines()  # cartopy function

### Faceting maps

We can make faceted maps. Since `FacetGrid` creates the axes it plots to, we
need to pass the `projection` kwarg in `subplot_kws`. This makes sure that the
subplots are set up properly for cartopy.


In [ ]:
fg = monthly_means.Tair.isel(month=[1, 2, 3]).plot(
    col="month",
    transform=ccrs.PlateCarree(),  # remember to provide this!
    subplot_kws={"projection": ccrs.LambertConformal(central_longitude=-95, central_latitude=45)},
    cbar_kwargs={"orientation": "horizontal", "shrink": 0.8, "aspect": 40},
    robust=True,
)

# lets add a coastline to each axis
# great reason to use FacetGrid.map
fg.map(lambda: plt.gca().coastlines())

<a id="bokeh"></a>

## Interactive bokeh plots using hvplot

Xarray's builtin plotting functionality wraps matplotlib.

The `holoviews` plotting ecosystem provides the `hvplot` package to allow easy
visualization of xarray (and other) objects (https://hvplot.holoviz.org/). These
plots build on [Bokeh](https://bokeh.org/).

`hvplot` makes uses of xarray's accessor interface. This means that all xarray
objects gain a `.hvplot` attribute that lets you access `hvplot` functionality
as easily as you would use `.plot`


In [ ]:
import hvplot.xarray

**`hvplot` makes the same default choices as xarray**


In [ ]:
ds.Tair.hvplot()

In [ ]:
# 2D array yields a quadmesh plot
ds.Tair.isel(time=1).hvplot(cmap="fire")

In [ ]:
# 1D array yields a line plot
ds.Tair.isel(time=1, lon=1).hvplot()

### Interactivity

But `hvplot` shines when interactivity is used. Here we can give it _all_ the
data and ask it to create a nice slider to control the time slice using the
`groupby` kwarg.


In [ ]:
ds.Tair.hvplot(
    groupby="time",
    clim=(250, 295),  # adds a widget for time  # sets colorbar limits
)

Animations are easy


In [ ]:
# set constant colorbar limits
ds.Tair.hvplot(
    groupby="time",  # adds a widget for time
    clim=(250, 295),  # sets colormap limits
    widget_type="scrubber",
    widget_location="bottom",
)

### Geography


In [ ]:
ds.Tair.isel(time=1).hvplot(
    projection=ccrs.Orthographic(-90, 30),
    coastline=True,
)

## More resources

1. Xarray's visualization gallery:
   https://xarray.pydata.org/en/stable/examples/visualization_gallery.html
2. Xarray's plotting documentation:
   https://xarray.pydata.org/en/stable/plotting.html
3. hvplot's plotting documentation:
   https://hvplot.holoviz.org/user_guide/Gridded_Data.html
